In [ ]:
!pip install ultralytics

In [3]:
import logging
import cv2
import time
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import os

logging.getLogger('ultralytics').setLevel(logging.WARNING)

# Yolo tracking

In [6]:
path_to_video = "cut_output_0.mp4"
detection_model = YOLO('tracker.pt')
video = cv2.VideoCapture(path_to_video)

# Настройка выходного видео
frame_width = int(video.get(3))
frame_height = int(video.get(4))

output_file = f'Yolo_tracker.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, 20.0, (frame_width, frame_height))

# Директория для сохранения вырезанных объектов

start = time.time()
frame_count = 0
while True:

    # Выводим каждый 20 кадр
    if frame_count % 20 == 0:
        print(f'Frame {frame_count}')

    frame_count += 1
    ret, frame = video.read()
    if not ret:
        break

    # Применяем модель YOLO для детекции объектов
    results = detection_model.track(frame, persist=True, conf=0.3)
    annotated_frame = results[0].plot()

    # Записываем кадр в выходное видео
    out.write(annotated_frame)


end = time.time()
video.release()
out.release()
cv2.destroyAllWindows()
print(f"\nСреднее время обработки одного кадра - {round((end - start) / frame_count, 4)}")

# Yolo classification

In [7]:
test_img = [
   "apple_test_1.png",
   "apple_test_2.png",
   "apple_test_3.png",
   "apple_test_4.png",
   "apple_test_5.png",
]

In [10]:
classification_model = YOLO('classification.pt')

for img_path in test_img:
    results = classification_model(img_path)

    annotated_img = results[0].plot()

    plt.imshow(cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()


# Full pipeline

In [ ]:
# Подключаем гугл диск
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def draw_custom_annotations(frame, results, tracker_d, save_dir, frame_count):
    output = []
    for obj in results[0].boxes:
        x1, y1, x2, y2 = map(int, obj.xyxy[0].tolist())

        D = max(abs(x2 - x1), abs(y2 - y1))
        track_obj = obj.id
        if not track_obj is None:
            track_id = track_obj.item()
            cropped_object = frame[y1:y2, x1:x2]

            # Классифицируем
            class_results = classification_model.predict(cropped_object, conf=0.25)
            class_label = class_results[0].names[np.argmax(class_results[0].probs.data).item()]
            probs =  max(class_results[0].probs.data).item()

            # Установите нужный текст для аннотации
            custom_text = f'Id {track_id} {class_label} {round(probs, 2)} D={round(D, 2)}'

            path_to_save = f'{save_dir}/apple_{track_id}'
            if track_id not in tracker_d:
                tracker_d[track_id] = {
                    'general': {
                        'count': 0,
                        'sum_bad': 0,
                        'sum_D': 0,
                        'sum_median_h': 0,
                    },
                    'history': []
                }
                os.makedirs(path_to_save, exist_ok=True)

            object_filename = os.path.join(path_to_save, f'id_{track_id}_label_{class_label}_frame_{frame_count}.jpg')
            cv2.imwrite(object_filename, cropped_object)


            hsv_frame = cv2.cvtColor(cropped_object, cv2.COLOR_BGR2HSV)
            h_channel, s_channel, v_channel = cv2.split(hsv_frame)

            # Вычисление медианы для каждого канала
            median_h = np.median(h_channel)
            median_s = np.median(s_channel)
            median_v = np.median(v_channel)

            tracker_d[track_id]['history'].append({
                'label': class_label,
                'prob': probs,
                'D': round(D, 2),
                'x1': x1,
                'x2': x2,
                'y1': y1,
                'y2': y2,
                'median_h': median_h,
                'median_s': median_s,
                'median_v': median_v
            })
            tracker_d[track_id]['general']['count'] += 1
            tracker_d[track_id]['general']['sum_bad'] += int(class_label != 'good')
            tracker_d[track_id]['general']['sum_D'] += round(D, 2)
            tracker_d[track_id]['general']['sum_median_h'] += median_h

            if y2 < 30:
                if x2 > 700:
                    tracker_d[track_id]['general']['line'] = 1
                else:
                    tracker_d[track_id]['general']['line'] = 2
                output.append(track_id)

            # Определите цвет и шрифт
            if class_label == "good":
                color = (0, 255, 0)
            else:
                color = (0, 0, 255)

            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5
            font_thickness = 1
            text_size, _ = cv2.getTextSize(custom_text, font, font_scale, font_thickness)
            text_x = x1
            text_y = y1 - 10 if y1 - 10 > 10 else y1 + text_size[1] + 10

            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, custom_text, (text_x, text_y), font, font_scale, color, font_thickness)
    return tracker_d, output

In [ ]:
path = "/content/drive/MyDrive/Проекты/apples"
detection_model = YOLO('/content/tracker.pt')
classification_model = YOLO('/content/classification.pt')
video = cv2.VideoCapture(f"{path}/Apples_class.mp4")

# Настройка выходного видео
frame_width = int(video.get(3))
frame_height = int(video.get(4))
output_file = f'{path}/Yolo_tracker.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_file, fourcc, 20.0, (frame_width, frame_height))

# Директория для сохранения вырезанных объектов
save_dir = f'cut_apples'
os.makedirs(save_dir, exist_ok=True)

start = time.time()
frame_count = 0
tracker_d = {}
while frame_count < 300:
    if frame_count % 20 == 0:
        end_local = time.time()
        print(frame_count, f'mean_time = {(end_local - start)  / (frame_count + 1)}')
    frame_count += 1
    ret, frame = video.read()
    if not ret:
        break

    # Применяем модель YOLO для детекции объектов
    results = detection_model.track(frame, persist=True, conf=0.3)
    tracker_d, output = draw_custom_annotations(frame, results, tracker_d, save_dir, frame_count)

    # Записываем кадр в выходное видео
    out.write(frame)
    for i in output:
        line = tracker_d[i]['general']['line']
        general_sum_bad = tracker_d[i]['general']['sum_bad']
        general_count = tracker_d[i]['general']['count']
        general_sum_D = tracker_d[i]['general']['sum_D']
        general_sum_median_h = tracker_d[i]['general']['sum_median_h']

        text = f"Яблоко {int(i)} вышло на {line} линии!\n" + ' ' * 4
        text += f"Плохое в {general_sum_bad}/{general_count} ({round(100 * general_sum_bad / general_count)}%) кадрах\n" + ' ' * 4
        text += f"Средний диаметр - {round(general_sum_D / general_count, 2)}\n" + ' ' * 4
        text += f"Средний h - {round(general_sum_median_h / general_count, 2)}\n" + ' ' * 4

        print(text)

end = time.time()
video.release()
out.release()
cv2.destroyAllWindows()
print(f"\nTime - {(end - start) / frame_count}")